In [1]:
import sys
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import label

# Add your custom library path
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/")

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, outliers_to_nan
from ieeg.io import raw_from_layout, get_data
# Assuming get_trials_with_outlier_analysis is in your general_utils
from src.analysis.utils.general_utils import get_trials_with_outlier_analysis

def outlier_analysis(subjects, event, times, outlier_threshold, create_outlier_plots, mark_outliers_as_nan, channels=None, task='GlobalLocal', LAB_root=None):
    """
    Performs outlier analysis for a list of subjects.
    """
    # Determine LAB_root once, outside the loop if it's not provided.
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    for sub in subjects:
        print(f'-----\nProcessing subject: {sub}')

        # The save_dir is specific to each subject, so it should be defined inside the loop
        save_dir = os.path.join(LAB_root, 'BIDS-1.1_GlobalLocal', 'BIDS', 'derivatives', 'tests', 'outlierAnalysis', sub)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Load data for the subject
        layout = get_data(task, root=LAB_root)
        filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                               extension='.edf', desc='clean', preload=False)

        good = crop_empty_data(filt)
        
        print(f"Channels before dropping bads: {len(good.ch_names)}")
        good.info['bads'] = channel_outlier_marker(good, 3, 2)
        print("Marked bad channels:", good.info['bads'])
        good.drop_channels(good.info['bads'])
        print(f"Channels after dropping bads: {len(good.ch_names)}")

        good.load_data()

        # Use a local variable for the channels to be picked for the current subject.
        channels_to_use = channels
        if channels_to_use is not None:
            # Validate the provided channels for the current subject
            invalid_channels = [ch for ch in channels_to_use if ch not in good.ch_names]
            if invalid_channels:
                raise ValueError(f"Subject {sub}: The following channels are not valid: {invalid_channels}")
            good.pick_channels(channels_to_use)

        # Re-referencing
        ch_type = good.get_channel_types(only_data_chs=True)[0]
        good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

        # Run the analysis and plotting
        trials = get_trials_with_outlier_analysis(
            data=good, 
            events=event, 
            times=times, 
            outlier_threshold=outlier_threshold, 
            create_outlier_plots=create_outlier_plots, 
            mark_outliers_as_nan=mark_outliers_as_nan, 
            save_dir=save_dir
        )
        print(f"Finished processing for {sub}. Plots saved in {save_dir}")

# ===================================================================
# Script Execution Starts Here
# ===================================================================

# Define all parameters for the analysis
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']
# subjects = ['D0057']
event_type = ['Stimulus']
times = (-1, 1.5)
outlier_threshold = 10
create_outlier_plots = True
mark_outliers_as_nan = True
task = 'GlobalLocal'

# Call the function with the correct, simplified set of arguments.
outlier_analysis(
    subjects=subjects, 
    event=event_type, 
    times=times, 
    outlier_threshold=outlier_threshold, 
    create_outlier_plots=create_outlier_plots, 
    mark_outliers_as_nan=mark_outliers_as_nan, 
    task=task
)

print("\nAll subjects processed!")

-----
Processing subject: D0057
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...
EDF file d

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected


...

Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0121\ieeg\sub-D0121_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0121\ieeg\sub-D0121_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0121\ieeg\sub-D0121_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 219 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0121\ieeg\sub-D0121_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Channels before dropping bads: 211
outlier round 1 channels: ['LTAS8']
outlier round 1 channels: ['LTAS8', 'LTMM15']
outlier round 1 channels: ['LTAS8', 'LTMM15', 'LFOP17']
outlier round 1 channels: ['LTAS8', 'LTMM15', 'LFOP17', 'LFPP8']
outlier round 1 channels: ['LTAS8', 'LTMM15', 'LFOP17', 'LFPP8', 'LFMI10']
outlier round 2 channels: ['LTAS8', 'LTMM15', 'LFOP17', 'LFPP8', 'LFMI10', 'LTAM15']
Marked bad channels: ['LTAS8', 'LTMM15', 'LFOP17', 'LFPP8', 'LFMI10', 'LTAM15']
Channels after dropping bads: 205
Reading 0 ... 6972696  =      0.000 ...  3404.637 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['Response/c25.0/n25.0/BigLetters/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse114.0/Co

c:\Users\jz421\Desktop\GlobalLocal\src\analysis\utils\general_utils.py:1666: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  all_trials = mne.concatenate_epochs(all_trials_list)


Not setting metadata
448 matching events found
No baseline correction applied

Total trials before outlier marking: 448
time range: -1.5 to 2.0

OUTLIER ANALYSIS

Per-Channel Statistics:
  Number of channels: 205
  Number of channels with outliers: 179
  For channels WITH outliers:
    Mean outlier trials: 4.4
    Median outlier trials: 2.0
    Max outlier trials: 29.0

  Top 5 worst channels: ['LFPP9', 'LTMM4', 'LTMM1', 'LTMM2', 'LTMM3']
  Their outlier times (ms): ['29.000', '28.000', '27.000', '26.000', '25.000']

--- Impact Assessment ---
If dropping outlier trials: 912/91840 channel-trials lost (1.0%)
Channels with >1% outlier trials: 68/205 channel-trials lost (33.2%)
Channels with >5% outliers: 6/205 channel-trials lost (2.9%)
Attempting to save plot to: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\tests\outlierAnalysis\D0121\Stimulus_outlier_analysis.png
Plot save command executed successfully.

Finished processing for D0121. Plots saved in C:\Users\jz421\B